In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from pylab import rcParams

In [ ]:
#import the files
df = pd.read_csv('file_directory.csv')
df

In [ ]:
#Read your file
df = pd.DataFrame(df)
df.rename({0:'Cases',1:'Temp_Max',2:'Humi_Rel',3:'Prec',4:'Pred_1'}, axis=1, inplace=True)
df.head()

In [ ]:
#Preparing your dataset according to the paper description
y = df['Pred_1']
X= df.drop(['Pred_1'],axis=1)
X= X.drop(['Temp_Max'],axis=1)
X= X.drop(['Prec'],axis=1)
#X= X.drop(['Humi_Rel'],axis=1)
X

In [ ]:
#use the normalization
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(X)
RobustScaler()
X = transformer.transform(X)
X = pd.DataFrame(X)

In [ ]:
#here you select the train size, for ex is set to be 0.76 of the whole time series
train_size= int(len(df)*0.76)
test_size = int(len(df)) - train_size + 1

In [ ]:
#Start the training
x_train, y_train = X[:train_size].dropna(), y[:train_size].dropna()
x_val, y_val = X[train_size:].dropna(), y[train_size:].dropna()

In [ ]:
X_train_series = x_train.values.reshape((x_train.shape[0], x_train.shape[1]))
X_valid_series = x_val.values.reshape((x_val.shape[0], x_val.shape[1]))

In [ ]:
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_train_series.shape)

In [ ]:
#Define the hyperparameters according to the city and the paper description
rf = RandomForestRegressor(criterion = 'poisson', n_estimators = 1500, max_features = 0.2, max_depth = 200,bootstrap=True, random_state = 18).fit(X_train_series, y_train)
prediction_train = rf.predict(X_train_series)
prediction_test = rf.predict(X_valid_series )

In [ ]:
y_true = df['Pred_1'].values
val = pd.DataFrame(y_val)
val["pred_test"] = prediction_test

In [ ]:
train = pd.DataFrame(y_train)
train["pred_train"] = prediction_train
pred = np.concatenate([prediction_train,prediction_test])
y_true = df['Pred_1'].values

In [ ]:
treino_teste = np.concatenate([prediction_train, prediction_test])

In [ ]:
from matplotlib import pyplot
importances = rf.feature_importances_

In [ ]:
y_true = pd.DataFrame(y_true)
y_true.rename({0:'Cases'}, axis=1, inplace=True)

In [ ]:
treino_teste = pd.DataFrame(treino_teste)
treino_teste.rename({0:'Cases'}, axis=1, inplace=True)

In [ ]:
from numpy import sqrt 
from sklearn.metrics import mean_squared_error
erro_abs= np.abs(y_true-treino_teste)
rmse_total = sqrt(mean_squared_error(y_true, treino_teste))

In [ ]:
y_true_data = pd.DataFrame(y_true)
simulado_data = pd.DataFrame(treino_teste)
erro_abs_data =  pd.DataFrame(erro_abs)
erro=y_true-treino_teste
result = pd.concat([y_true_data, simulado_data, erro_abs_data,erro], axis=1)
result.to_csv('result.csv')

In [ ]:
#Compute the error
from math import sqrt
from sklearn.metrics import r2_score
from scipy import stats
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_percentage_error
erro_medio = np.mean(erro_abs[train_size:train_size+test_size])
rmse = sqrt(mean_squared_error(y_true[train_size:], treino_teste[train_size:]))
r2 = r2_score(y_true[train_size:], treino_teste[train_size:])
xx = y_true_data[train_size:]
xx = xx.rename(columns={'Cases': 'Cases'})
yy = treino_teste[train_size:]
yy = yy.rename(columns={'Cases': 'Treino'})
dff = pd.concat([xx, yy], axis=1)
corr, _ = pearsonr(dff['Cases'],dff['Treino'])
####
#print('Erro Perc. Medio: %.3f' % error_percentual_medio)
print('Erro abs: %.3f' %erro_medio)
print('Pearsons correlation: %.3f' % corr)
print('Test RMSE: %.3f' % rmse)
print('R^2: %.3f' % r2)
#print('Corr: %.3f' % r)
importances